# DQN mountaincar - main model

In [28]:
import gym
import numpy as np
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.optimizers import Adam
from keras.layers.convolutional import Conv1D, MaxPooling1D

from collections import deque

class DQN:
    def __init__(self, env):
        self.env     = env
        self.memory  = deque(maxlen=500) # appends to queue and pops at the other end
        
        self.gamma = 0.85
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.02
        self.tau = .125

        self.model        = self.create_model()
        self.target_model = self.create_model()

    def create_model(self):
        model   = Sequential()
        state_shape  = self.env.observation_space.shape
        model.add(Dense(128, input_dim=state_shape[0], activation='relu'))
        model.add(BatchNormalization())
        model.add(Dense(64, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dense(self.env.action_space.n))
        model.compile(loss="mean_squared_error",
            optimizer=Adam(lr=self.learning_rate))
        return model

    # take action
    def act(self, state):
        self.epsilon *= self.epsilon_decay
        self.epsilon = max(self.epsilon_min, self.epsilon)
        if np.random.random() < self.epsilon:
            return self.env.action_space.sample()
        return np.argmax(self.model.predict(state)[0])

    # remember history
    def remember(self, state, action, reward, new_state, done):
        self.memory.append([state, action, reward, new_state, done])

        
    def replay(self):
        batch_size = 64
        if len(self.memory) < batch_size: 
            return

        samples = random.sample(self.memory, batch_size)
        for sample in samples:
            state, action, reward, new_state, done = sample
            target = self.target_model.predict(state)
            if done:
                target[0][action] = reward
            else:
                Q_future = max(self.target_model.predict(new_state)[0])
                target[0][action] = reward + Q_future * self.gamma
            self.model.fit(state, target, epochs=1, verbose=0)

    def target_train(self):
        weights = self.model.get_weights()
        target_weights = self.target_model.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = weights[i] * self.tau + target_weights[i] * (1 - self.tau)
        self.target_model.set_weights(target_weights)

    def save_model(self, fn):
        self.model.save(fn)

env     = gym.make("MountainCar-v0")
gamma   = 0.95
epsilon = .95
trials  = 1000
trial_len = 500

keep_result = []
dqn_agent = DQN(env=env)
steps = []
for trial in range(trials):
    cur_state = env.reset().reshape(1,2)
    for step in range(trial_len):
        # take action from current state (epsilon greedy)
        action = dqn_agent.act(cur_state)
        # take one step
        new_state, reward, done, _ = env.step(action)

        # reward = reward if not done else -20
        new_state = new_state.reshape(1,2)
        # store in memory
        dqn_agent.remember(cur_state, action, reward, new_state, done)
        
        dqn_agent.replay()       # internally iterates default (prediction) model

        dqn_agent.target_train() # iterates target model

        cur_state = new_state
        if done:
            break
    if step >= 199:
        keep_result.append(200)
        print("Failed to complete in trial {}".format(trial))
        if step % 10 == 0:
            dqn_agent.save_model("trial-{}.model".format(trial))
    else:
        keep_result.append(step)
        print("Completed in {} trials".format(trial))
        dqn_agent.save_model("success.model")
        

#main()


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Failed to complete in trial 0
Failed to complete in trial 1
Failed to complete in trial 2
Failed to complete in trial 3
Failed to complete in trial 4
Failed to complete in trial 5
Failed to complete in trial 6
Failed to complete in trial 7
Failed to complete in trial 8
Failed to complete in trial 9
Failed to complete in trial 10
Failed to complete in trial 11
Failed to complete in trial 12
Failed to complete in trial 13
Failed to complete in trial 14
Failed to complete in trial 15
Failed to complete in trial 16
Failed to complete in trial 17
Failed to complete in trial 18
Failed to complete in trial 19
Failed to complete in trial 20
Failed to complete in trial 21
Failed to complete in trial 22
Failed to complete in trial 23
Failed to complete in trial 24
Failed to complete in trial 25
Failed to complete in trial 26
Failed to complete in trial 27
Failed to complete in trial 28
Failed to co

KeyboardInterrupt: 

In [29]:
keep_result

[200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 192,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 183,
 200,
 200,
 118,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 123,
 200,
 200,
 119,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200

# DQN Mountaincar - old model

In [ ]:
import gym
import numpy as np
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.optimizers import Adam
from keras.layers.convolutional import Conv1D, MaxPooling1D

from collections import deque

class DQN:
    def __init__(self, env):
        self.env     = env
        self.memory  = deque(maxlen=500) # appends to queue and pops at the other end
        
        self.gamma = 0.85
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.02
        self.tau = .125

        self.model        = self.create_model()
        self.target_model = self.create_model()

    def create_model(self):
        model   = Sequential()
        state_shape  = self.env.observation_space.shape
        model.add(Dense(16, input_dim=state_shape[0], activation="relu"))
        model.add(Dense(32, activation="relu"))
        model.add(Dense(64, activation="relu"))
        model.add(Dense(self.env.action_space.n))
        model.compile(loss="mean_squared_error",
            optimizer=Adam(lr=self.learning_rate))
        return model

    # take action
    def act(self, state):
        self.epsilon *= self.epsilon_decay
        self.epsilon = max(self.epsilon_min, self.epsilon)
        if np.random.random() < self.epsilon:
            return self.env.action_space.sample()
        return np.argmax(self.model.predict(state)[0])

    # remember history
    def remember(self, state, action, reward, new_state, done):
        self.memory.append([state, action, reward, new_state, done])

        
    def replay(self):
        batch_size = 64
        if len(self.memory) < batch_size: 
            return

        samples = random.sample(self.memory, batch_size)
        for sample in samples:
            state, action, reward, new_state, done = sample
            target = self.target_model.predict(state)
            if done:
                target[0][action] = reward
            else:
                Q_future = max(self.target_model.predict(new_state)[0])
                target[0][action] = reward + Q_future * self.gamma
            self.model.fit(state, target, epochs=1, verbose=0)

    def target_train(self):
        weights = self.model.get_weights()
        target_weights = self.target_model.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = weights[i] * self.tau + target_weights[i] * (1 - self.tau)
        self.target_model.set_weights(target_weights)

    def save_model(self, fn):
        self.model.save(fn)

#def main():
env     = gym.make("MountainCar-v0")
gamma   = 0.95
epsilon = .95
trials  = 1000
trial_len = 500

# updateTargetNetwork = 1000
keep_result2 = []
dqn_agent = DQN(env=env)
steps = []
for trial in range(trials):
    cur_state = env.reset().reshape(1,2)
    for step in range(trial_len):
        # take action from current state (epsilon greedy)
        action = dqn_agent.act(cur_state)
        # take one step
        new_state, reward, done, _ = env.step(action)

        # reward = reward if not done else -20
        new_state = new_state.reshape(1,2)
        # store in memory
        dqn_agent.remember(cur_state, action, reward, new_state, done)
        
        dqn_agent.replay()       # internally iterates default (prediction) model

        dqn_agent.target_train() # iterates target model

        cur_state = new_state
        if done:
            break
    if step >= 199:
        keep_result2.append(200)
        print("Failed to complete in trial {}".format(trial))
        if step % 10 == 0:
            dqn_agent.save_model("trial-{}.model".format(trial))
    else:
        keep_result2.append(step)
        print("Completed in {} trials".format(trial))
        dqn_agent.save_model("success.model")
        

#main()


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Failed to complete in trial 0
Failed to complete in trial 1
Failed to complete in trial 2
Failed to complete in trial 3
Failed to complete in trial 4
Failed to complete in trial 5
Failed to complete in trial 6
Failed to complete in trial 7
Failed to complete in trial 8
Failed to complete in trial 9
Failed to complete in trial 10
Failed to complete in trial 11
Failed to complete in trial 12
Failed to complete in trial 13
Failed to complete in trial 14
Failed to complete in trial 15
Failed to complete in trial 16
Failed to complete in trial 17
Failed to complete in trial 18
Failed to complete in trial 19
Failed to complete in trial 20
Failed to complete in trial 21
Failed to complete in trial 22
Failed to complete in trial 23
Failed to complete in trial 24
Failed to complete in trial 25
Failed to complete in trial 26
Failed to complete in trial 27
Failed to complete in trial 28
Failed to co

In [ ]:
keep_result2